In [2]:
import glob
import json
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import re
import operator 
from collections import Counter
from nltk import bigrams
import pandas as pd
import csv
import unicodecsv

In [3]:
# Removing unexpected new lines
with open('crypto_tweets.json') as infile, open('crypto_tweetsNoL.json', 'w') as outfile:
    i = -1;
    for line in infile:
        if not line.strip(): continue  # skip the empty line
        outfile.write(line)  # non-empty line. Write it to output

In [5]:
with open('crypto_tweetsNoL.json', 'r') as f:
    lines = f.readlines() # read only the first tweet/line
    tweet = json.loads(lines[0]) # load it as Python dict
    print(json.dumps(tweet, indent=4)) # pretty-print

{
    "created_at": "Sun Feb 18 17:55:02 +0000 2018",
    "id": 965283531667787776,
    "id_str": "965283531667787776",
    "text": "RT @CoinomiWallet: Announcing support for $LCC Litecoin Cash, $LTC's upcoming fork. To receive your free $LCC make sure you keep your $LTC\u2026",
    "source": "<a href=\"http://twitter.com\" rel=\"nofollow\">Twitter Web Client</a>",
    "truncated": false,
    "in_reply_to_status_id": null,
    "in_reply_to_status_id_str": null,
    "in_reply_to_user_id": null,
    "in_reply_to_user_id_str": null,
    "in_reply_to_screen_name": null,
    "user": {
        "id": 745181736129069056,
        "id_str": "745181736129069056",
        "name": "Decent Ape",
        "screen_name": "decentape",
        "location": "Denver, CO",
        "url": null,
        "description": "HODL so hard you forget you have it",
        "translator_type": "none",
        "protected": false,
        "verified": false,
        "followers_count": 39,
        "friends_count": 214,
     

In [6]:
tweet = json.loads(lines[1]) # load it as Python dict
jt = json.dumps(tweet) # pretty-print
tweet

{'contributors': None,
 'coordinates': None,
 'created_at': 'Sun Feb 18 17:55:02 +0000 2018',
 'entities': {'hashtags': [{'indices': [22, 27], 'text': 'IOTA'}],
  'symbols': [],
  'urls': [],
  'user_mentions': [{'id': 77346069,
    'id_str': '77346069',
    'indices': [3, 15],
    'name': 'Jake Williams',
    'screen_name': 'MalwareJake'}]},
 'favorite_count': 0,
 'favorited': False,
 'filter_level': 'low',
 'geo': None,
 'id': 965283532754087936,
 'id_str': '965283532754087936',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'is_quote_status': False,
 'lang': 'en',
 'place': None,
 'quote_count': 0,
 'reply_count': 0,
 'retweet_count': 0,
 'retweeted': False,
 'retweeted_status': {'contributors': None,
  'coordinates': None,
  'created_at': 'Sat Feb 17 18:20:30 +0000 2018',
  'display_text_range': [0, 140],
  'entities': {'hashtags': [{'indices': [5, 10], 'text': 'I

In [22]:
len(lines)

268151

In [23]:
# Removing unnecessary words and characters
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + [':/','com',':D','RT','rt', 'via', 'RT', 'I', 'We', 'After', 'The', 'the', '...', 'amp', 'Our', 'accept', 'A', 'FL', 'krab','https','@aoojadi','took','This','us','like','You','put','bathtub','…']

tweet = 'RT @marcobonzanini: just an example! :D http://example.com #NLP I ...'
print(preprocess(tweet))

terms_all = [term for term in preprocess(tweet) if term not in stop]
print(terms_all)

['marcobonzanini', 'just', 'example', ':D', 'http', ':/', 'example', 'com', 'NLP']
['marcobonzanini', 'example', 'http', 'example', 'NLP']


In [8]:
f = open("crypto_tweetsNoL.json", "r")
lines = f.readlines()
f.close
i=0
for line in lines:
    if i==38196 or 40150:
        i=i+1
        continue
    tweet = json.loads(line)
    i= i+1;
print(i)

1439


In [30]:
fname = 'crypto_tweets2NoL.json'
with open(fname, 'r') as f:
    count_all = Counter()
    count_bigram = Counter()
    i = -1
    for line in f:
        i=i+1
        if i>500000:
            break
        elif i==38196:
            continue
        elif i==40150:
            continue
        elif i>=268148:
            break
        else:
            #print(line)
            tweet = json.loads(line)
            keys = tweet.keys()
            if 'text' in tweet:
                #print(i)
                # Create a list with all the terms
                terms_all = [term for term in preprocess(tweet['text']) if term not in stop]
                terms_bigram = bigrams(terms_all)
                # Update the counter
                count_all.update(terms_all)
                count_bigram.update(terms_bigram)
    # Print the first 5 most frequent words
    print(count_all.most_common(20))
    print(count_bigram.most_common(20))

[('Bitcoin', 55635), ('BTC', 36612), ('global', 21561), ('BTG', 20950), ('cryptocurrency', 20263), ('still', 20056), ('exchanges', 19891), ('launch', 19680), ('start', 19530), ('Gold', 19365), ('Successfully', 18987), ('mainnet', 18879), ('gaining', 18723), ('popularity', 18699), ('becoming', 18662), ('trades', 18616), ('expensive', 18322), ('bitcoin', 18229), ('balance', 17967), ('crypto', 17599)]
[(('Successfully', 'launch'), 18869), (('launch', 'mainnet'), 18845), (('Bitcoin', 'Gold'), 18794), (('mainnet', 'global'), 18774), (('Gold', 'BTG'), 18734), (('global', 'exchanges'), 18725), (('BTG', 'gaining'), 18689), (('exchanges', 'start'), 18663), (('gaining', 'popularity'), 18633), (('popularity', 'becoming'), 18536), (('start', 'trades'), 18502), (('becoming', 'expensive'), 18282), (('expensive', 'still'), 17845), (('trades', 'balance'), 17836), (('balance', 'BTC'), 16060), (('claim', 'free'), 6157), (('Use', 'link'), 6118), (('ClassyCoin', 'claim'), 6040), (('free', 'Classycoin'), 6

In [15]:
with open('crypto_tweetsNoL.json') as infile, open('sample_1000_tweets.json', 'w') as outfile:
    i = 1;
    for line in infile:
        if not line.strip(): continue  # skip the empty line
        outfile.write(line)  # non-empty line. Write it to output
        if i>999:
            break
        i+=1

In [16]:
f = open("sample_1000_tweets.json", "r")
lines = f.readlines()
f.close
i=0
for line in lines:
    if i==38196 or 40150:
        i=i+1
        continue
    tweet = json.loads(line)
    i= i+1;
print(i)

1000
